# Census Data Tools

Initiate and configure the logger. This will save the outputs of the log to the specified file. 

In [1]:
from morpc.logs import config_logs

config_logs('./temp_data/morpc-census-demo.log', level='info') ## Designate the logger level. 

2025-11-10 17:16:33,749 | INFO | morpc.logs.config_logs: Set up logging save to file ./temp_data/morpc-census-demo.log


## morpc.census.api

This module has been developed to fetch and save Census data from various API endpoints. 

## Census API Class

The class CensusAPI class does most of the heavy lifting, formulating the API call, getting the data, wrangling it into a long format, creating frictionless schema and resource files, and saving the data. 

The class takes the following parameters:

1. The "survey/table" to get the data from. 
2. The year of the data. 
3. The group of variables that you want to call.
4. The geographic "scope" of the data you want to get the data for.

Optionally:

5. The geographic scale of the data you want to get the data for.
6. A subset of the variables in the chosen group.



In [2]:
# TODO: Timeseries calls.  

# TODO: Getting select variables from various groups.

## Example: Compare divorced population for all tracts in 15-County region.

### 1. See currently implemented endpoints for "survey/table" parameter.

Use the IMPLEMENTED_ENDPOINTS constant to see currently available options.

In [3]:
from morpc.census.api import IMPLEMENTED_ENDPOINTS

IMPLEMENTED_ENDPOINTS

2025-11-10 17:16:36,821 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data with parameters None.


['acs/acs1',
 'acs/acs1/profile',
 'acs/acs1/subject',
 'acs/acs5',
 'acs/acs5/profile',
 'acs/acs5/subject',
 'dec/pl',
 'dec/dhc',
 'dec/ddhca',
 'dec/ddhcb',
 'dec/sf1',
 'dec/sf2',
 'dec/sf3']

### 2. See available vintages

To see all available endpoints provided by the Census Bureau and their available vintages, see ALL_AVAIL_ENPOINTS

In [4]:
from morpc.census.api import ALL_AVAIL_ENDPOINTS 

ALL_AVAIL_ENDPOINTS['acs/acs5']

[2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023]

### 3. See available variable groups for a survey/table.

In [5]:
from morpc.census.api import get_table_groups

{k: v['description'] for k, v in get_table_groups('acs/acs5', 2023).items()}

# TODO: Hierarchical lookup for variables of interest.

{'B01001': 'Sex by Age',
 'B01001A': 'Sex by Age (White Alone)',
 'B01001B': 'Sex by Age (Black or African American Alone)',
 'B01001C': 'Sex by Age (American Indian and Alaska Native Alone)',
 'B01001D': 'Sex by Age (Asian Alone)',
 'B01001E': 'Sex by Age (Native Hawaiian and Other Pacific Islander Alone)',
 'B01001F': 'Sex by Age (Some Other Race Alone)',
 'B01001G': 'Sex by Age (Two or More Races)',
 'B01001H': 'Sex by Age (White Alone, Not Hispanic or Latino)',
 'B01001I': 'Sex by Age (Hispanic or Latino)',
 'B01002': 'Median Age by Sex',
 'B01002A': 'Median Age by Sex (White Alone)',
 'B01002B': 'Median Age by Sex (Black or African American Alone)',
 'B01002C': 'Median Age by Sex (American Indian and Alaska Native Alone)',
 'B01002D': 'Median Age by Sex (Asian Alone)',
 'B01002E': 'Median Age by Sex (Native Hawaiian and Other Pacific Islander Alone)',
 'B01002F': 'Median Age by Sex (Some Other Race Alone)',
 'B01002G': 'Median Age by Sex (Two or More Races)',
 'B01002H': 'Median A

###  4. Select the geographic scope of the data.

To see currently implemented scopes. See morpc.census.geos.SCOPES

In [6]:
from morpc.census.geos import SCOPES

SCOPES

# TODO: Passing list of geoids to ucgid parameter in api call

{'us': {'for': 'us:1'},
 'region15': {'in': 'state:39',
  'for': 'county:041,045,049,089,097,129,159,083,101,117,047,073,091,127,141'},
 'region10': {'in': 'state:39',
  'for': 'county:041,045,049,089,097,129,159,083,101,117'},
 'region7': {'in': 'state:39', 'for': 'county:041,045,049,089,097,129,159'},
 'region-corpo': {'in': 'state:39',
  'for': 'county:045,083,097,101,117,129,159'},
 'alabama': {'for': 'state:01'},
 'alaska': {'for': 'state:02'},
 'arizona': {'for': 'state:04'},
 'arkansas': {'for': 'state:05'},
 'california': {'for': 'state:06'},
 'colorado': {'for': 'state:08'},
 'connecticut': {'for': 'state:09'},
 'delaware': {'in': 'state:39', 'for': 'county:041'},
 'district of columbia': {'for': 'state:11'},
 'florida': {'for': 'state:12'},
 'georgia': {'for': 'state:13'},
 'hawaii': {'for': 'state:15'},
 'idaho': {'for': 'state:16'},
 'illinois': {'for': 'state:17'},
 'indiana': {'for': 'state:18'},
 'iowa': {'for': 'state:19'},
 'kansas': {'for': 'state:20'},
 'kentucky': {

### (Optional) 5. Select a smaller geographic scale to get data for. 

For example if you want to get all the tracts in the 15-county MORPC region.

To see available combinations of scope and scale see morpc.census.geos.PSUEDOS.

The key is the parent sumlevel and the values represent the child sumlevel to get the data for.

In [7]:
from morpc.census.geos import PSEUDOS

PSEUDOS

## All available scale levels if the sumlevel of the "scope" paramter is Counties (050)

PSEUDOS['050']

['0600000',
 '06V0000',
 '1000000',
 '1400000',
 '1500000',
 '1600000',
 '7000000',
 '8600000',
 '8710000']

### (Optional) 6. A list of variables to get from the group.

In [2]:
from morpc.census.api import get_group_variables

{k: v['label'] for k, v in get_group_variables('acs/acs5', 2023, 'B12001').items()}

{'B12001_001E': 'Estimate!!Total:',
 'B12001_001EA': 'Annotation of Estimate!!Total:',
 'B12001_001M': 'Margin of Error!!Total:',
 'B12001_001MA': 'Annotation of Margin of Error!!Total:',
 'B12001_002E': 'Estimate!!Total:!!Male:',
 'B12001_002EA': 'Annotation of Estimate!!Total:!!Male:',
 'B12001_002M': 'Margin of Error!!Total:!!Male:',
 'B12001_002MA': 'Annotation of Margin of Error!!Total:!!Male:',
 'B12001_003E': 'Estimate!!Total:!!Male:!!Never married',
 'B12001_003EA': 'Annotation of Estimate!!Total:!!Male:!!Never married',
 'B12001_003M': 'Margin of Error!!Total:!!Male:!!Never married',
 'B12001_003MA': 'Annotation of Margin of Error!!Total:!!Male:!!Never married',
 'B12001_004E': 'Estimate!!Total:!!Male:!!Now married:',
 'B12001_004EA': 'Annotation of Estimate!!Total:!!Male:!!Now married:',
 'B12001_004M': 'Margin of Error!!Total:!!Male:!!Now married:',
 'B12001_004MA': 'Annotation of Margin of Error!!Total:!!Male:!!Now married:',
 'B12001_005E': 'Estimate!!Total:!!Male:!!Now ma

In [7]:
variables =['B12001_001E', 'B12001_002E', 'B12001_010E', 'B12001_011E', 'B12001_019E']

### Get the data by calling CensusAPI

In [8]:
from morpc.census.api import CensusAPI

b12001_2023 = CensusAPI('acs/acs5', 2023, 'B12001', 'region15', variables=variables)

The raw data is available and stored in the class as .DATA

In [9]:
b12001_2023.DATA

,GEO_ID,NAME,B12001_001E,B12001_002E,B12001_010E,B12001_011E,B12001_019E
0,0500000US39041,"Delaware County, Ohio",175941,87452,6483,88489,9367
1,0500000US39045,"Fairfield County, Ohio",129785,64395,6340,65390,8845
2,0500000US39047,"Fayette County, Ohio",23391,11464,1571,11927,1806
3,0500000US39049,"Franklin County, Ohio",1061441,516471,48846,544970,69884
4,0500000US39073,"Hocking County, Ohio",22983,11460,1631,11523,1679
5,0500000US39083,"Knox County, Ohio",50940,24881,2444,26059,2825
6,0500000US39089,"Licking County, Ohio",146195,71645,7360,74550,9532
7,0500000US39091,"Logan County, Ohio",37494,18625,2095,18869,2291
8,0500000US39097,"Madison County, Ohio",36828,20457,3190,16371,2584
9,0500000US39101,"Marion County, Ohio",53588,28990,4574,24598,3701


The class also automatically creates a long table format which is the format we will save the data as. 

In [10]:
b12001_2023.LONG

,GEO_ID,NAME,reference_period,variable_label,variable,estimate
0,0500000US39041,"Delaware County, Ohio",2023,Total:,B12001_001,175941
3,0500000US39041,"Delaware County, Ohio",2023,Total:!!Male:,B12001_002,87452
4,0500000US39041,"Delaware County, Ohio",2023,Total:!!Male:!!Divorced,B12001_010,6483
1,0500000US39041,"Delaware County, Ohio",2023,Total:!!Female:,B12001_011,88489
2,0500000US39041,"Delaware County, Ohio",2023,Total:!!Female:!!Divorced,B12001_019,9367
...,...,...,...,...,...,...
70,0500000US39159,"Union County, Ohio",2023,Total:,B12001_001,52356
73,0500000US39159,"Union County, Ohio",2023,Total:!!Male:,B12001_002,25061
74,0500000US39159,"Union County, Ohio",2023,Total:!!Male:!!Divorced,B12001_010,2158
71,0500000US39159,"Union County, Ohio",2023,Total:!!Female:,B12001_011,27295


#### morpc.census.api.DimensionTable

The other class that is implemented is the dimension table. These are typically used for situations in which you want the data to be human readable. 

This is designed to replicate how the Census often represents it's data with the different levels of the variable group as rows and the geography and years as columns. 

In [11]:
from morpc.census.api import DimensionTable

dim_table = DimensionTable(b12001_2023.LONG)

In [13]:
dim_table.wide().T

0                                                        Total:          \
1                                                                 Male:   
2                                                                         
GEO_ID         NAME                   reference_period                    
0500000US39041 Delaware County, Ohio  2023               175941   87452   
0500000US39045 Fairfield County, Ohio 2023               129785   64395   
0500000US39047 Fayette County, Ohio   2023                23391   11464   
0500000US39049 Franklin County, Ohio  2023              1061441  516471   
0500000US39073 Hocking County, Ohio   2023                22983   11460   
0500000US39083 Knox County, Ohio      2023                50940   24881   
0500000US39089 Licking County, Ohio   2023               146195   71645   
0500000US39091 Logan County, Ohio     2023                37494   18625   
0500000US39097 Madison County, Ohio   2023                36828   20457   
0500000US39101 Marion County, Ohio    2023                53588   28990   
0500000US39117 Morrow County, Ohio    2023                28955   14546   
0500000US39127 Perry County, Ohio     2023                28661   14319   
0500000US39129 Pickaway County, Ohio  2023                49013   25999   
0500000US39141 Ross County, Ohio      2023                63306   33229   
0500000US39159 Union County, Ohio     2023                52356   25061   

0                                                                        \
1                                                               Female:   
2                                                      Divorced           
GEO_ID         NAME                   reference_period                    
0500000US39041 Delaware County, Ohio  2023                 6483   88489   
0500000US39045 Fairfield County, Ohio 2023                 6340   65390   
0500000US39047 Fayette County, Ohio   2023                 1571   11927   
0500000US39049 Franklin County, Ohio  2023                48846  544970   
0500000US39073 Hocking County, Ohio   2023                 1631   11523   
0500000US39083 Knox County, Ohio      2023                 2444   26059   
0500000US39089 Licking County, Ohio   2023                 7360   74550   
0500000US39091 Logan County, Ohio     2023                 2095   18869   
0500000US39097 Madison County, Ohio   2023                 3190   16371   
0500000US39101 Marion County, Ohio    2023                 4574   24598   
0500000US39117 Morrow County, Ohio    2023                 1742   14409   
0500000US39127 Perry County, Ohio     2023                 1926   14342   
0500000US39129 Pickaway County, Ohio  2023                 2970   23014   
0500000US39141 Ross County, Ohio      2023                 4314   30077   
0500000US39159 Union County, Ohio     2023                 2158   27295   

0                                                                
1                                                                
2                                                      Divorced  
GEO_ID         NAME                   reference_period           
0500000US39041 Delaware County, Ohio  2023                 9367  
0500000US39045 Fairfield County, Ohio 2023                 8845  
0500000US39047 Fayette County, Ohio   2023                 1806  
0500000US39049 Franklin County, Ohio  2023                69884  
0500000US39073 Hocking County, Ohio   2023                 1679  
0500000US39083 Knox County, Ohio      2023                 2825  
0500000US39089 Licking County, Ohio   2023                 9532  
0500000US39091 Logan County, Ohio     2023                 2291  
0500000US39097 Madison County, Ohio   2023                 2584  
0500000US39101 Marion County, Ohio    2023                 3701  
0500000US39117 Morrow County, Ohio    2023                 1760  
0500000US39127 Perry County, Ohio     2023                 1872  
0500000US39129 Pickaway County, Ohio  2023                 3038  
0500000US39141 Ross County

Also implemented is the percent table. Which shows the data as a percentage of the total (ie. univierse)

In [14]:
dim_table.percent().T

0                                                          Total:            \
1                                                           Male:             
2                                                                  Divorced   
GEO_ID         NAME                   reference_period                        
0500000US39041 Delaware County, Ohio  2023              49.705299  3.684758   
0500000US39045 Fairfield County, Ohio 2023              49.616674  4.885002   
0500000US39047 Fayette County, Ohio   2023              49.010303  6.716258   
0500000US39049 Franklin County, Ohio  2023              48.657533  4.601857   
0500000US39073 Hocking County, Ohio   2023              49.862942  7.096550   
0500000US39083 Knox County, Ohio      2023              48.843738  4.797801   
0500000US39089 Licking County, Ohio   2023              49.006464  5.034372   
0500000US39091 Logan County, Ohio     2023              49.674615  5.587561   
0500000US39097 Madison County, Ohio   2023              55.547410  8.661888   
0500000US39101 Marion County, Ohio    2023              54.097932  8.535493   
0500000US39117 Morrow County, Ohio    2023              50.236574  6.016232   
0500000US39127 Perry County, Ohio     2023              49.959876  6.719933   
0500000US39129 Pickaway County, Ohio  2023              53.045110  6.059617   
0500000US39141 Ross County, Ohio      2023              52.489495  6.814520   
0500000US39159 Union County, Ohio     2023              47.866529  4.121782   

0                                                                            
1                                                         Female:            
2                                                                  Divorced  
GEO_ID         NAME                   reference_period                       
0500000US39041 Delaware County, Ohio  2023              50.294701  5.323944  
0500000US39045 Fairfield County, Ohio 2023              50.383326  6.815117  
0500000US39047 Fayette County, Ohio   2023              50.989697  7.720918  
0500000US39049 Franklin County, Ohio  2023              51.342467  6.583880  
0500000US39073 Hocking County, Ohio   2023              50.137058  7.305400  
0500000US39083 Knox County, Ohio      2023              51.156262  5.545740  
0500000US39089 Licking County, Ohio   2023              50.993536  6.520059  
0500000US39091 Logan County, Ohio     2023              50.325385  6.110311  
0500000US39097 Madison County, Ohio   2023              44.452590  7.016401  
0500000US39101 Marion County, Ohio    2023              45.902068  6.906397  
0500000US39117 Morrow County, Ohio    2023              49.763426  6.078398  
0500000US39127 Perry County, Ohio     2023              50.040124  6.531524  
0500000US39129 Pickaway County, Ohio  2023              46.954890  6.198356  
0500000US39141 Ross County, Ohio      2023              47.510505  7.397403  
0500000US39159 Union County, Ohio     2023              52.133471  5.516082

### Timeseries of calls

Dimension table takes any long format table that is formatted similar to the output of CensusAPI. 

This allows for easy implementation of timeseries data by concatenating two long form tables from different years.

In [15]:
b12001_2018 = CensusAPI('acs/acs5', 2018, 'B12001', 'region15', scale='tract', variables=variables)

In [16]:
import pandas as pd

long = pd.concat([b12001_2023.LONG, b12001_2018.LONG])

In [17]:
dim_table = DimensionTable(long)

In [18]:
percent = dim_table.percent()

In [20]:
percent.T

0                                                                                  Total  \
1                                                                                   Male   
2                                                                                          
GEO_ID               NAME                                    reference_period              
0500000US39041       Delaware County, Ohio                   2023              49.705299   
0500000US39045       Fairfield County, Ohio                  2023              49.616674   
0500000US39047       Fayette County, Ohio                    2023              49.010303   
0500000US39049       Franklin County, Ohio                   2023              48.657533   
0500000US39073       Hocking County, Ohio                    2023              49.862942   
...                                                                                  ...   
1400000US39159050400 Census Tract 504, Union County, Ohio    2018              33.276031   
1400000US39159050500 Census Tract 505, Union County, Ohio    2018              44.049248   
1400000US39159050601 Census Tract 506.01, Union County, Ohio 2018              49.366774   
1400000US39159050602 Census Tract 506.02, Union County, Ohio 2018              57.115118   
1400000US39159050700 Census Tract 507, Union County, Ohio    2018              52.057714   

0                                                                                        \
1                                                                                         
2                                                                              Divorced   
GEO_ID               NAME                                    reference_period             
0500000US39041       Delaware County, Ohio                   2023              3.684758   
0500000US39045       Fairfield County, Ohio                  2023              4.885002   
0500000US39047       Fayette County, Ohio                    2023              6.716258   
0500000US39049       Franklin County, Ohio                   2023              4.601857   
0500000US39073       Hocking County, Ohio                    2023              7.096550   
...                                                                                 ...   
1400000US39159050400 Census Tract 504, Union County, Ohio    2018              4.715128   
1400000US39159050500 Census Tract 505, Union County, Ohio    2018              6.611947   
1400000US39159050601 Census Tract 506.01, Union County, Ohio 2018              3.625528   
1400000US39159050602 Census Tract 506.02, Union County, Ohio 2018              6.629681   
1400000US39159050700 Census Tract 507, Union County, Ohio    2018              3.164453   

0                                                                                         \
1                                                                                 Female   
2                                                                                          
GEO_ID               NAME                                    reference_period              
0500000US39041       Delaware County, Ohio                   2023              50.294701   
0500000US39045       Fairfield County, Ohio                  2023              50.383326   
0500000US39047       Fayette County, Ohio                    2023              50.989697   
0500000US39049       Franklin County, Ohio                   2023              51.342467   
0500000US39073       Hocking County, Ohio                    2023              50.137058   
...                                                                                  ...   
1400000US39159050400 Census Tract 504, Union County, Ohio    2018              66.723969   
1400000US39159050500 Census Tract 505, Union County, Ohio    2018              55.950752   
1400000US39159050601 Census Tract 506.01, Union County, Ohio 2018              50.633226   
1400000US39159050602 Census Tract 506.02, Union County, O

## (DEPRECIATED) Custom API calls

### Getting all available datasets in the api.

MORPC works regularly with census data, including but not limited to ACS 5 and 1-year, Decennial Census, PEP, and geographies. The following module is useful for gathering and organizing census data for processes in various workflow. Those workflows are linked when appropriate. 

## API functions and variables

api_get() is a low-level wrapper for Census API requests that returns the results as a pandas dataframe. If necessary, it splits the request into several smaller requests to bypass the 50-variable limit imposed by the API.  

The resulting dataframe is indexed by GEOID (regardless of whether it was requested) and omits other fields that are not requested but which are returned automatically with each API request (e.g. "state", "county") 

In [22]:
url = 'https://api.census.gov/data/2022/acs/acs1'
params = {
    "get": "GEO_ID,NAME,B01001_001E",
    "for": "county:049,041",
    "in": "state:39"
}

In [24]:
from morpc.census.api import get
api = get(url, params)

In [25]:
api

,GEO_ID,NAME,B01001_001E
0,0500000US39041,"Delaware County, Ohio",226296
1,0500000US39049,"Franklin County, Ohio",1321820


## Geography tools

In [1]:
from morpc.census import geos

In [2]:
geos.SCOPES['region15']

{'in': 'state:39',
 'for': 'county:041,045,049,089,097,129,159,083,101,117,047,073,091,127,141'}

In [3]:
geos.fetch_geos_from_geoids(['0500000US39049'], '2023', 'ACS')


,geometry
GEO_ID,
0500000US39049,"POLYGON ((-83.21025 40.06927, -83.21028 40.069..."


In [8]:
from morpc.census.api import geo_params_from_scope_scale
from morpc.census.geos import geoids_from_pseudo, fetch_geos_from_geoids

fetch_geos_from_geoids(geoids_from_pseudo(geo_params_from_scope_scale('region15', 'block group').values()), '2023', 'ACS')

# TODO: Split geoid list into sumlevels and then chucks to avoid to large of requests. Figure out how to handle this in the rest_api module maybe.

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))